In [1]:
import warnings
warnings.filterwarnings("ignore")

import DataUtils.MATH as MATH
import DataUtils.MMLU as MMLU
from hfUtils import *
# from ModelUtils import *
from vLLMUtils import *

import numpy as np

import torch

In [2]:
model = load_model('meta-llama/Llama-3.2-1B-Instruct')

WARNING 11-30 12:44:35 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-30 12:44:35 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-30 12:44:35 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=39808, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=au

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 11-30 12:44:37 model_runner.py:1067] Loading model weights took 2.3185 GB
INFO 11-30 12:44:38 gpu_executor.py:122] # GPU blocks: 35947, # CPU blocks: 8192
INFO 11-30 12:44:38 gpu_executor.py:126] Maximum concurrency for 39808 tokens per request: 14.45x
INFO 11-30 12:44:39 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-30 12:44:39 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-30 12:44:45 model_runner.py:1523] Graph capturing finished in 7 secs.


In [16]:
MMLU_inputs_chat, MMLU_Answers_chat = MMLU.get_test_data_chat_template()
MATH_inputs_chat, MATH_Answers_chat = MATH.get_test_data_chat_template()

MMLU_inputs_plain, MMLU_Answers_plain = MMLU.get_test_data_plain()
MATH_inputs_plain, MATH_Answers_plain = MATH.get_test_data_plain()

MATH_inputs_chat = model.get_tokenizer().apply_chat_template(MATH_inputs_chat, add_generation_prompt=True, tokenize=False)

In [4]:
# print some sample tokenized inputs
rnd_idx = np.random.randint(0, len(MATH_inputs_chat))
print("MATH Chat Template Input: \n", MATH_inputs_chat[rnd_idx])
print("MATH Plain Input: \n", MATH_inputs_plain[rnd_idx])

MATH Chat Template Input: 
 [{'role': 'user', 'content': 'How many numbers can be expressed as the sum of two or more distinct elements of the set $\\{ 0, 1, 2, 4, 8, 16 \\}$?'}]
MATH Plain Input: 
 Below is an instruction that describes a task. Write a response that appropriately completes the request. Think step by step but be concise. Decorate the final answer with \boxed{}

### Instruction:
How many numbers can be expressed as the sum of two or more distinct elements of the set $\{ 0, 1, 2, 4, 8, 16 \}$?

### Response: Let's think step by step.


In [17]:
sampling_params = MATH.get_sampling_params()

In [18]:
sampling_params

SamplingParams(n=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.0, top_p=1.0, top_k=-1, min_p=0.0, seed=None, stop=[], stop_token_ids=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=4096, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None), guided_decoding=None

In [ ]:
outputs = model.generate(MATH_inputs_chat, sampling_params=sampling_params)

Processed prompts:   0%|          | 4/5000 [00:03<47:41,  1.75it/s, est. speed input: 78.26 toks/s, output: 170.99 toks/s] 

In [20]:
results = extract_outputs(outputs)

In [21]:
answers = MATH.extract_answers(results)

In [22]:
coorect = 0
for i in range(len(answers)):
    if answers[i].strip() == MATH_Answers_chat[i].strip():
        coorect += 1

print("Accuracy: ", coorect/len(answers))

Accuracy:  0.11


In [23]:
answers

['\\begin{pmatrix} 6 & 2 \\\\ -1 & -3 \\end{pmatrix}',
 '\\frac{\\pi}{2}',
 '## Step 1: Recall the trigonometric identity for the sum of cosines\nWe know that $\\cos A + \\cos B = 2\\cos\\left(\\frac{A+B}{2}\\right)\\cos\\left(\\frac{A-B}{2}\\right)$.\n\n## Step 2: Apply the identity to the given equation\nUsing the identity, we can rewrite the given equation as $\\cos x + \\cos 2x + \\cos 3x = 2\\cos\\left(\\frac{x+2x}{2}\\right)\\cos\\left(\\frac{x-2x}{2}\\right) + \\cos 3x = 2\\cos\\left(\\frac{3x}{2}\\right)\\cos(-x) + \\cos 3x = 2\\cos\\left(\\frac{3x}{2}\\right)\\cos x + \\cos 3x$.\n\n## Step 3: Simplify the equation\nSince $\\cos(-x) = \\cos x$, we can simplify the equation to $2\\cos\\left(\\frac{3x}{2}\\right)\\cos x + \\cos 3x = 3$.\n\n## Step 4: Recall the trigonometric identity for the sum of sines\nWe know that $\\sin A + \\sin B = 2\\sin\\left(\\frac{A+B}{2}\\right)\\cos\\left(\\frac{A-B}{2}\\right)$.\n\n## Step 5: Apply the identity to the given equation\nUsing the ident